In [1]:
#COLAB - mount drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:

#PROTOTYPE CODE - IMAGE ANALYSIS FOR PAINTINGS GENERATED BY OPEN SOURCE STYLEGAN MODELS
#by JON THUM


!git clone https://github.com/jonthum/AiPainter
%cd AiPainter

#COLAB - root directory
ROOT_DIR = '/content/AiPainter/'

#IMAGE ANALYSIS DIRECTORY
DATA_DIR = ROOT_DIR + 'custom/imageanalysis/'


#CHECK SETTINGS
%tensorflow_version 1.x
import tensorflow as tf
print('Tensorflow version: {}'.format(tf.__version__) )

!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

from psutil import virtual_memory
ram = virtual_memory().total / 1e9
print('RAM: {:.1f} gigabytes\n'.format(ram))


#CLONE STYLEGAN CODE
!git clone https://github.com/NVlabs/stylegan
%cd stylegan

#DOWNLOAD OPEN SOURCE GENERIC MODEL network-snapshot-008040.pkl
!gdown --id 1zC28MQuBcdhpYJCFqGj-bHgKFQc4UqW1

#DOWNLOAD OPEN SOURCE PORTRAIT MODEL network-snapshot-011125.pkl
!gdown --id 1cJQtMeTy_QldOP7n64F8stCDXY6Esup9

#MODEL LIBRARIES
import ipywidgets as widgets
import pickle
import dnnlib
import dnnlib.tflib as tflib

#LOAD GENERIC MODEL
network_pkl = 'network-snapshot-008040.pkl'

tflib.init_tf()
model = open(network_pkl, 'rb')
_G, _D, Gs1 = pickle.load(model, encoding='latin1')

#LOAD PORTRAIT MODEL
network_pkl = 'network-snapshot-011125.pkl'

tflib.init_tf()
model = open(network_pkl, 'rb')
_G, _D, Gs2 = pickle.load(model, encoding='latin1')

#DEFAULT ARGUMENTS
Gs_syn_kwargs = dnnlib.EasyDict()
Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
Gs_syn_kwargs.minibatch_size = 1
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

#IMPORT LIBRARIES
import os
import os.path
import time
import numpy as np
from PIL import Image, ImageFilter, ImageStat, ImageChops, ImageMath
import statistics as st


fatal: destination path 'AiPainter' already exists and is not an empty directory.
/content/AiPainter
TensorFlow 1.x selected.
Tensorflow version: 1.15.2
GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-f86357d3-56fd-f806-4c16-592d0ce034cb)
GPU Identified at: /device:GPU:0
RAM: 13.7 gigabytes

fatal: destination path 'stylegan' already exists and is not an empty directory.
/content/AiPainter/stylegan
Downloading...
From: https://drive.google.com/uc?id=1zC28MQuBcdhpYJCFqGj-bHgKFQc4UqW1
To: /content/AiPainter/stylegan/network-snapshot-008040.pkl
308MB [00:01, 230MB/s]
Downloading...
From: https://drive.google.com/uc?id=1cJQtMeTy_QldOP7n64F8stCDXY6Esup9
To: /content/AiPainter/stylegan/network-snapshot-011125.pkl
308MB [00:01, 262MB/s]













Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
#BASIC INTERACTIVE PAINTING GENERATION


#WIDGETS
model = widgets.Dropdown(options=[('GENERIC', True), ('PORTRAIT', False)], value=True, description='MODEL: ')
rand = widgets.IntSlider(min=0, max=5000, step=1, value=0, description='Rand: ', continuous_update=False)
truncation = widgets.FloatSlider(min=-2, max=2, step=0.1, value=1, description='Trunc: ')
button = widgets.Button(description='Save')
ui = widgets.HBox([model, rand, truncation, button])

#GENERATE IMAGES
def f(model, rand, truncation):
    global image
    if (model):
        Gs = Gs1
    else:
        Gs = Gs2
    rnd = np.random.RandomState(rand)
    l1 = rnd.randn(1, Gs.input_shape[1]) #512
    images = Gs.run(l1, None, truncation_psi=truncation, randomize_noise=False, output_transform=fmt)
    image = Image.fromarray(images[0])
    display(image)

#DISPLAY
out = widgets.interactive_output(f, {'model': model, 'rand': rand, 'truncation': truncation})
display(ui, out)

#SAVE IMAGE
def save_image(b):
    name = 'Painting_{}_{}.jpg'.format(int(model.value), rand.value)
    path = ROOT_DIR + 'images/results/' + name
    image.save(path)       
    print('SAVED {}'.format(name))

button.on_click(save_image)

Output()

SAVED Painting_1_2153.jpg


In [4]:
#IMAGE ANALYSIS PROTOTYPE - ONLY NEED TO RUN ONCE TO CREATE DATAFILE


#RUN ON GENERIC MODEL
Gs = Gs1

#HSV COLOUR HUES  
colour = {'red': 255, 'yellow': 42, 'green': 85, 'cyan': 127, 'blue':170, 'magenta': 212}
centre = 0

#KEY HUE VALUES
def hue_key(x):
    global centre
    radius = 43     #max 127
    
    #Shift everything to origin at 128 to avoid negative numbers    
    y = (x-centre+128)%255       
    key = int(3*(radius - abs(y-128)))
    key = max(key, 0)   
    
    return key


#MEAN VALUE FOR SPECIFIED COLOUR = HUE KEY x SAT x VAL
def colour_stat(hue, sat, val):
    key = Image.eval(hue, hue_key)
    col = ImageMath.eval("convert(a*b/255, 'L')", a=key, b=sat)
    colval = ImageMath.eval("convert(a*b/255, 'L')", a=col, b=val)
    stat = ImageStat.Stat(colval)
    
    return stat.mean[0]


#GENERATE IMAGE FROM MODEL
def generate_painting(seed):

    rnd = np.random.RandomState(seed)
    l1 = rnd.randn(1, Gs.input_shape[1])    
    images = Gs.run(l1, None, truncation_psi=1, randomize_noise=False, output_transform=fmt)

    return images[0]


#INIT
start = time.process_time()
print('Processing ....')

#INCLUSIVE RANGE
START = 0
END = 5000

datafile = []


#LOOP THROUGH IMAGES
for id in range(START, END+1):

    #PROGRESS FEEDBACK
    if(id%500 == 0):
        print('{} images:'.format(id), time.process_time() - start)
    
    #GENERATE PAINTING
    painting = generate_painting(id)
    image = Image.fromarray(painting)

    #DETAIL ANALYSIS
    grey = image.convert('L')
    blur = grey.filter(ImageFilter.GaussianBlur(10))
    diff = ImageChops.difference(grey, blur)
    
    D = 4   #Normalising constant
    stat = ImageStat.Stat(diff)
    detail = int(D*stat.mean[0])
    
    
    #DOWNSIZE TO SAVE COMPUTATION
    image = image.resize((128,128))
  
    
    #HSV ANALYSIS
    hsv = image.convert(mode="HSV")
    
    hue = hsv.getchannel('H')
    sat = hsv.getchannel('S')
    val = hsv.getchannel('V')
  
    stat = ImageStat.Stat(hsv)
    brightness = int(stat.mean[2]/2)
    contrast = int(stat.stddev[2])
    saturation = int(stat.mean[1]/2)
    

    #INDIVIDUAL COLOUR ANALYSIS
    C = 8   #Normalising constant
    
    centre = colour['red']
    red_mean = int(C*colour_stat(hue, sat, val))
    
    centre = colour['yellow']
    yellow_mean = int(C*colour_stat(hue, sat, val))
    
    centre = colour['green']
    green_mean = int(C*colour_stat(hue, sat, val))
    
    centre = colour['cyan']
    cyan_mean = int(C*colour_stat(hue, sat, val))
    
    centre = colour['blue']
    blue_mean = int(C*colour_stat(hue, sat, val))
    
    centre = colour['magenta']
    magenta_mean = int(C*colour_stat(hue, sat, val))


    #COLOUR VARIETY
    T = 60   #Max value for completeness
    
    col_means = [red_mean, yellow_mean, green_mean, cyan_mean, blue_mean, magenta_mean]
    mean = st.mean(col_means)
    stdev = st.stdev(col_means)
    total = min(red_mean, T) + min(yellow_mean, T) + min(green_mean, T) + \
            min(cyan_mean, T) + min(blue_mean, T) + min(magenta_mean, T)
    variety = int(total*mean/(stdev+1)/2)    
            
    #APPEND DATA
    info = [detail, brightness, contrast, saturation, variety, 
            red_mean, yellow_mean, green_mean, 
            cyan_mean, blue_mean, magenta_mean, id]
    #print(info)
    datafile.append(info)        
    

#SAVE DATA
datafile = np.array(datafile, dtype=np.int)
name = 'Prototype_analysis.npy'
np.save(DATA_DIR + name, datafile)
print(datafile.shape)

print('END:', time.process_time() - start)










Processing ....
0 images: 0.0005225230000007741
500 images: 18.416742731000003
1000 images: 35.05592278100001
1500 images: 51.80034901399999
2000 images: 68.384355579
2500 images: 85.22982847899999
3000 images: 101.949468013
3500 images: 118.82683337499999
4000 images: 135.587268819
4500 images: 152.434914969
5000 images: 169.44786188100002
(5001, 12)
END: 169.491015511


In [5]:
#FILTERING UI PROTOTYPE


#RUN ON GENERIC MODEL
Gs = Gs1

#PARAMETERS
SEED = 0
SIZE = 3

#LOAD IMAGE ANALYSIS FILE
datafile = np.load(DATA_DIR + 'Prototype_analysis.npy')

#TESTING
SAMPLES = 5001
datafile = datafile[:SAMPLES]


#INTERACTIVE MENUS
seed = widgets.IntSlider(min=0, max=5000, step=1, value=SEED, description='ID: ', continuous_update=False)
truncation = widgets.FloatSlider(min=-2, max=2, step=0.01, value=1, description='Trunc: ', continuous_update=False)
noise = widgets.Dropdown(options=[('On', True), ('Off', False)], value=False, description='Noise: ')
size = widgets.Dropdown(options=[('Full', 3), ('Half', 2), ('Quarter', 1)], value=SIZE, description='Display: ')
filt = widgets.Dropdown(options=[('On', True), ('Off', False)], value=True, description='Filter: ')
add_noise = widgets.ToggleButton(description='Add Noise')
save = widgets.Button(description='Save')
det = widgets.IntRangeSlider(value=[0, 100], min=0, max=100, step=1, description='Detail:', continuous_update=False)
bri = widgets.IntRangeSlider(value=[0, 100], min=0, max=100, step=1, description='Brightness:', continuous_update=False)
con = widgets.IntRangeSlider(value=[0, 100], min=0, max=100, step=1, description='Contrast:', continuous_update=False)
sat = widgets.IntRangeSlider(value=[0, 100], min=0, max=100, step=1, description='Saturation:', continuous_update=False)
var = widgets.IntRangeSlider(value=[0, 100], min=0, max=100, step=1, description='Variety:', continuous_update=False)
R = widgets.IntRangeSlider(value=[0, 100], min=0, max=100, step=1, description='R:', continuous_update=False)
Y = widgets.IntRangeSlider(value=[0, 100], min=0, max=100, step=1, description='Y:', continuous_update=False)
G = widgets.IntRangeSlider(value=[0, 100], min=0, max=100, step=1, description='G:', continuous_update=False)
C = widgets.IntRangeSlider(value=[0, 100], min=0, max=100, step=1, description='C:', continuous_update=False)
B = widgets.IntRangeSlider(value=[0, 100], min=0, max=100, step=1, description='B:', continuous_update=False)
M = widgets.IntRangeSlider(value=[0, 100], min=0, max=100, step=1, description='M:', continuous_update=False)
sample = widgets.IntSlider(value=1, min=1, max=5001, step=1, description='Sample #:', continuous_update=False)

#MENU LAYOUT
box1 = widgets.VBox([det, bri, con, sat, var, sample])   #widgets.Label(value="FILTERS"),
box2 = widgets.VBox([R, Y, G, C, B, M])
box3 = widgets.VBox([noise, size, filt, seed, truncation])
box4 = widgets.VBox([add_noise, save])
ui = widgets.HBox([box1, box2, box3, box4])


#MODEL ARGS
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)


#GENERATE PAINTING
def display_sample(seed, truncation, noise, add_noise, size, det, bri, con, sat, var, R, Y, G, C, B, M, sample, filt):
    global image, seed_id, num

    #DISPLAY RESOLUTION
    resolution = 64*(2**size)

    if(filt):
        #FILTER VALUES
        f = ([det[0], det[1]], [bri[0], bri[1]], [con[0], con[1]], [sat[0], sat[1]], [var[0], var[1]],
            [R[0], R[1]], [Y[0], Y[1]], [G[0], G[1]], [C[0], C[1]], [B[0], B[1]], [M[0], M[1]])

        #APPLY FILTERS
        df = datafile.copy() 
        for i in range(0,11):
            #MAX VAL NEEDS TO INCLUDE ALL
            if f[i][1]==100:
                f[i][1] = 999
            df = df[np.logical_and( df[:,i]>=f[i][0], df[:,i]<=f[i][1] )]
          
        #NUMBER OF PICKS IN RANGE
        num = df.shape[0]    

        if(num):
            #IDX OF PICKS
            pick = df[:,-1]

            #SHUFFLE
            shuffle = pick.copy()
            np.random.seed(0)
            np.random.shuffle(shuffle)

            #GET SAMPLE
            if(sample<num):
                seed_id = shuffle[sample-1]
            else:
                seed_id = shuffle[num-1]

            #FEEDBACK      
            print('\n  ID: {}   FILTERED SAMPLES: {}\n'.format(seed_id, num))

        else:
            seed_id = seed
            print('\n  ID: {}   NO SAMPLES IN RANGE\n'.format(seed_id))

    else:
        seed_id = seed
        print('\n  ID: {}   FILTERING OFF\n'.format(seed_id))       

    #GENERATE IMAGE FROM SEED
    rnd = np.random.RandomState(seed_id)
    l1 = rnd.randn(1, Gs.input_shape[1])    
    images = Gs.run(l1, None, truncation_psi=truncation, randomize_noise=noise, output_transform=fmt)
    image = Image.fromarray(images[0])    
    display_image = image.resize((resolution, resolution))
    display(display_image)

#MENU DISPLAY
out = widgets.interactive_output(display_sample, {'seed': seed, 'truncation': truncation, 
                                                  'noise': noise, 'add_noise': add_noise, 'size': size, 
                                                  'det': det, 'bri': bri, 'con': con, 'sat': sat, 'var': var,
                                                  'R': R, 'Y': Y, 'G': G, 'C': C, 'B': B, 'M': M, 
                                                  'sample': sample, 'filt': filt})
out.layout.height = '600px'
display(ui, out)


#UPDATE SAMPLE MAX
def update_sample_max(change):
    sample.max = max(1, num)
det.observe(update_sample_max, names='value')
bri.observe(update_sample_max, names='value')
con.observe(update_sample_max, names='value')
sat.observe(update_sample_max, names='value')
var.observe(update_sample_max, names='value')
R.observe(update_sample_max, names='value')
Y.observe(update_sample_max, names='value')
G.observe(update_sample_max, names='value')
C.observe(update_sample_max, names='value')
B.observe(update_sample_max, names='value')
M.observe(update_sample_max, names='value')


#SAVE IMAGE
def save_image(b):
    name = 'Painting_1_{}.jpg'.format(seed_id)
    path = ROOT_DIR + 'images/results/' + name
    image.save(path)       
    print('SAVED {}'.format(name))

save.on_click(save_image)


Output(layout=Layout(height='600px'))

SAVED Painting_1_1804.jpg


In [6]:
#LATENT SPACE ANIMATION - SIMPLE INTERPOLATION


#RUN ON GENERIC MODEL
Gs = Gs1

#PICK TWO IMAGES AND ANIMATION LENGTH
RAND1 = 0
RAND2 = 7
FRAMES = 240

rnd = np.random.RandomState(RAND1)
latent_vector1 = rnd.randn(1, Gs.input_shape[1])

rnd = np.random.RandomState(RAND2)
latent_vector2 = rnd.randn(1, Gs.input_shape[1])


#DIRECTORIES
out_dir = ROOT_DIR + 'images/animation/'
out_name = 'interp_{}_{}'.format(RAND1, RAND2)

#MAKE TEMP DIR
temp_dir = ROOT_DIR + 'images/animation/temp'
os.system('mkdir "%s"'%temp_dir)
print(temp_dir)

#PROCESS IMAGES
number_of_frames = FRAMES
frame_step = 1.0/number_of_frames
x = 0
for frame_count in range(1,number_of_frames):
    x = x + frame_step
    latent_input = latent_vector1.copy()
    for i in range(512):
        f1 = latent_vector1[0][i]
        f2 = latent_vector2[0][i]
        fnew = f1 + (f2-f1)*x
        latent_input[0][i] = fnew

    #SAVE IMAGES
    images = Gs.run(latent_input, None, truncation_psi=1, randomize_noise=False, output_transform=fmt)
    name = temp_dir + '/animation_' + str(frame_count) + '.jpg'
    Image.fromarray(images[0], 'RGB').save(name) 

#MAKE VIDEO
cmd = 'ffmpeg -i "%s/animation_%%d.jpg" -c:v libx264 "%s%s.mp4"' % (temp_dir, out_dir, out_name) 
print(cmd)
os.system(cmd)

#DELETE TEMP DIR
os.system('rm -rf "%s"'%temp_dir)


/content/AiPainter/images/animation/temp
ffmpeg -i "/content/AiPainter/images/animation/temp/animation_%d.jpg" -c:v libx264 "/content/AiPainter/images/animation/interp_0_7.mp4"


0